In [2]:
import numpy as np
import networkx as nx
import netgraph
import matplotlib.pylab as plt
import os
from tqdm import tqdm

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.samplers.regular.leiden_sampler import LeidenSampler
from Qommunity.samplers.regular.louvain_sampler import LouvainSampler
from Qommunity.samplers.regular.dqm_sampler import DQMSampler
#from Qommunity.samplers.regular.bayan_sampler import BayanSampler

from Qommunity.searchers.community_searcher.community_searcher import CommunitySearcher
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher

from iterative_searcher.iterative_searcher import IterativeSearcher

type_net = 'ER' #'BA' # 'Power-law' #  'DSF' #
NUM_NETWORKS = 1
MIN_NODES = 10
MAX_NODES = 100
num_nodes = np.linspace(MIN_NODES, MAX_NODES, MAX_NODES//MIN_NODES)
resolution = 1
num_runs = 20
os.makedirs(f"./{type_net}", exist_ok=True)

# Network generation

In [3]:
# Properties of the networks
if type_net=='ER':
    p = 0.4 # Probability of edge creation
elif type_net=='BA':
    m = 0.4 # Fraction of nodes
elif type_net=='Power-law':
    m, p = 1, .1
elif type_net=='DSF':
    pass
else:
    raise ValueError("Network not implemented")

# Generation of the networks
if type_net=='ER':
    try:
        Graphs = np.load(f"./{type_net}/graphs-{NUM_NETWORKS}_p-{p}.npy", allow_pickle=True)
    except:
        Graphs = np.empty(shape=(len(num_nodes),), dtype=object)
        for i, n in enumerate(num_nodes):
            Graphs[i] = [nx.erdos_renyi_graph(n=int(n), p=p) for _ in range(NUM_NETWORKS)]
        np.save(f"./{type_net}/graphs-{NUM_NETWORKS}_p-{p}.npy", Graphs)        

elif type_net=='BA':
    try:
        Graphs = np.load(f"./{type_net}/graphs-{NUM_NETWORKS}_m-{m}.npy", allow_pickle=True)
    except:
        Graphs = np.empty(shape=(len(num_nodes),), dtype=object)
        for i, n in enumerate(num_nodes):
            Graphs[i] = [nx.barabasi_albert_graph(n=n, m=int(n*m)) for _ in range(NUM_NETWORKS)]
        np.save(f"./{type_net}/graphs-{NUM_NETWORKS}_m-{m}.npy", Graphs)

elif type_net=='Power-law':
    try:
        Graphs = np.load(f"./{type_net}/graphs-{NUM_NETWORKS}_m-{m}_p-{p}.npy", allow_pickle=True)
    except:
        Graphs = np.empty(shape=(len(num_nodes),), dtype=object)
        for i, n in enumerate(num_nodes):
            Graphs[i] = [nx.powerlaw_cluster_graph(n=n, m=m, p=p) for _ in range(NUM_NETWORKS)]
        np.save(f"./{type_net}/graphs-{NUM_NETWORKS}_m-{m}_p-{p}.npy", Graphs)

elif type_net=='DSF':
    pass

else:
    raise ValueError("Network not implemented")

# Community structure

In [4]:
### ADVANTAGE ANNEALING ###
adv_modularities = np.zeros((num_nodes.shape[0], NUM_NETWORKS))
adv_communities = np.empty((num_nodes.shape[0], NUM_NETWORKS), dtype=object)
adv_times_elapsed = np.zeros((num_nodes.shape[0], NUM_NETWORKS))

for i, G_nodes in enumerate(num_nodes):
    try:
        adv_modularities[i] = np.load(f"./{type_net}/solver-adv_runs-{num_runs}_nodes-{G_nodes}_modularities.npy")
        adv_communities[i] = np.load(f"./{type_net}/solver-adv_runs-{num_runs}_nodes-{G_nodes}_communities.npy", allow_pickle=True)
        adv_times_elapsed[i] = np.load(f"./{type_net}/solver-adv_runs-{num_runs}_nodes-{G_nodes}_times.npy")
        print(f"Reasults loaded for n={G_nodes}")
    except:        
        for j, G in enumerate(tqdm(Graphs[i], desc=f"Networks with {G_nodes} nodes completed:")):
            adv_sampler = AdvantageSampler(G, resolution=resolution, num_reads=100, use_clique_embedding=True)
            adv_searcher = HierarchicalCommunitySearcher(adv_sampler)
            adv_iterative= IterativeSearcher(adv_searcher)
            cs_all, mod_all, ts_all = adv_iterative.run(num_runs=num_runs, save_results=False)

            # Only the community with the highest modularity
            adv_communities[i,j] = cs_all[mod_all.argmax()]
            adv_modularities[i,j] = mod_all.max()
            adv_times_elapsed[i,j] = ts_all[mod_all.argmax()]

        np.save(f"./{type_net}/solver-adv_runs-{num_runs}_nodes-{G_nodes}_modularities.npy", adv_modularities[i])
        np.save(f"./{type_net}/solver-adv_runs-{num_runs}_nodes-{G_nodes}_communities.npy", adv_communities[i], allow_pickle=True)
        np.save(f"./{type_net}/solver-adv_runs-{num_runs}_nodes-{G_nodes}_times.npy", adv_times_elapsed[i])

Reasults loaded for n=10.0
Reasults loaded for n=20.0
Reasults loaded for n=30.0
Reasults loaded for n=40.0
Reasults loaded for n=50.0
Reasults loaded for n=60.0
Reasults loaded for n=70.0
Reasults loaded for n=80.0


Networks with 90.0 nodes completed::   0%|          | 0/1 [00:00<?, ?it/s]

Networks with 100.0 nodes completed:: 100%|██████████| 1/1 [05:04<00:00, 304.63s/it]


In [5]:
### LOUVAIN ###
try:
    louv_modularities = np.load(f"./{type_net}/solver-louv_runs-{num_runs}_modularities.npy")
    louv_communities = np.load(f"./{type_net}/solver-louv_runs-{num_runs}_communities.npy", allow_pickle=True)
    louv_times_elapsed = np.load(f"./{type_net}/solver-louv_runs-{num_runs}_times.npy")
    print("Reasults loaded")
except:
    louv_modularities = np.zeros((num_nodes.shape[0], NUM_NETWORKS))
    louv_communities = np.empty((num_nodes.shape[0], NUM_NETWORKS), dtype=object)
    louv_times_elapsed = np.zeros((num_nodes.shape[0], NUM_NETWORKS))

    for i, G_nodes in enumerate(num_nodes):
        for j, G in enumerate(tqdm(Graphs[i], desc=f"Networks with {G_nodes} nodes completed:")):
            louv_sampler = LouvainSampler(G, resolution=resolution)
            louv_searcher = CommunitySearcher(louv_sampler)
            louv_iterative= IterativeSearcher(louv_searcher)
            cs_all, mod_all, ts_all = louv_iterative.run(num_runs=num_runs, save_results=False)

            # Only the community with the highest modularity
            louv_communities[i,j] = cs_all[mod_all.argmax()]
            louv_modularities[i,j] = mod_all.max()
            louv_times_elapsed[i,j] = ts_all[mod_all.argmax()]

    np.save(f"./{type_net}/solver-louv_runs-{num_runs}_modularities.npy", louv_modularities)
    np.save(f"./{type_net}/solver-louv_runs-{num_runs}_communities.npy", louv_communities, allow_pickle=True)
    np.save(f"./{type_net}/solver-louv_runs-{num_runs}_times.npy", louv_times_elapsed)

Networks with 100.0 nodes completed:: 100%|██████████| 1/1 [00:00<00:00,  1.41it/s]


In [6]:
### LEIDEN ###
try:
    leid_modularities = np.load(f"./{type_net}/solver-leid_runs-{num_runs}_modularities.npy")
    leid_communities = np.load(f"./{type_net}/solver-leid_runs-{num_runs}_communities.npy", allow_pickle=True)
    leid_times_elapsed = np.load(f"./{type_net}/solver-leid_runs-{num_runs}_times.npy")
    print("Reasults loaded")
except:
    leid_modularities = np.zeros((num_nodes.shape[0], NUM_NETWORKS))
    leid_communities = np.empty((num_nodes.shape[0], NUM_NETWORKS), dtype=object)
    leid_times_elapsed = np.zeros((num_nodes.shape[0], NUM_NETWORKS))

    for i, G_nodes in enumerate(num_nodes):
        for j, G in enumerate(tqdm(Graphs[i], desc=f"Networks with {G_nodes} nodes completed:")):
            leid_sampler = LeidenSampler(G)
            leid_searcher = CommunitySearcher(leid_sampler)
            leid_iterative= IterativeSearcher(leid_searcher)
            cs_all, mod_all, ts_all = leid_iterative.run(num_runs=num_runs, save_results=False)

            # Only the community with the highest modularity
            leid_communities[i,j] = cs_all[mod_all.argmax()]
            leid_modularities[i,j] = mod_all.max()
            leid_times_elapsed[i,j] = ts_all[mod_all.argmax()]

    np.save(f"./{type_net}/solver-leid_runs-{num_runs}_modularities.npy", leid_modularities)
    np.save(f"./{type_net}/solver-leid_runs-{num_runs}_communities.npy", leid_communities, allow_pickle=True)
    np.save(f"./{type_net}/solver-leid_runs-{num_runs}_times.npy", leid_times_elapsed)

Networks with 100.0 nodes completed:: 100%|██████████| 1/1 [00:00<00:00,  3.17it/s]


In [7]:
leid_times_elapsed[:5], louv_times_elapsed[:5], adv_times_elapsed[:5]

(array([[0.00836682],
        [0.00184011],
        [0.00159335],
        [0.00329041],
        [0.0040009 ]]),
 array([[0.00831127],
        [0.00213552],
        [0.00193143],
        [0.0094347 ],
        [0.00843358]]),
 array([[ 6.64670587],
        [ 6.78401995],
        [13.08130765],
        [14.32917356],
        [18.37536287]]))